In [1]:
#libraries
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)


### Levanto ENMODO 2014
http://datar.info/dataset/enmodo-2014

In [2]:
hogares = pd.read_csv('hogares.csv', delimiter='|')
personas = pd.read_csv('personas.csv', delimiter='|')
viajes = pd.read_csv('viajes.csv', delimiter='|')
etapas = pd.read_csv('etapas.csv', delimiter='|')

hogares = hogares.drop_duplicates().reset_index(drop=True)
personas = personas.drop_duplicates().reset_index(drop=True)
viajes = viajes.drop_duplicates().reset_index(drop=True)
etapas = etapas.drop_duplicates().reset_index(drop=True)

hogares.f_expansion.sum().round(0)
personas.f_expansion.sum().round(0)

hogares.loc[hogares.cod_provincia==2, 'Zona'] = 'CABA'
hogares.loc[hogares.cod_provincia==6, 'Zona'] = 'Conurbano'

# Codifico motivo destino de viajes y creo variable motivos agregada
viajes['Motivo'] = viajes.bivi06_02.replace([1,2,3,4,5,6,7,8,9,10,11,12,13], ['Hogar', 'Lugar de trabajo', 'Actividades relacionadas con el trabajo', 'Establecimiento educativo',
                'Actividades relacionadas con el estudio', 'Acompañar a un miembro del hogar a un establecimiento educativo', 
                'Acompañar a un miembro del hogar a otro lugar', 'Acompañar a un miembro de otro hogar', 'Compras',
                'Salud', 'Trámites', 'Ocio', 'Otros'])


viajes['Motivo_agg'] = viajes.Motivo.replace(['Hogar', 'Lugar de trabajo', 'Actividades relacionadas con el trabajo', 'Establecimiento educativo', 'Actividades relacionadas con el estudio', 
 'Acompañar a un miembro del hogar a un establecimiento educativo', 'Acompañar a un miembro del hogar a otro lugar', 'Acompañar a un miembro de otro hogar', 'Compras',
                'Salud', 'Trámites', 'Ocio', 'Otros'], ['Hogar', 'Trabajo', 'Trabajo', 'Estudio', 'Estudio', 'Cuidado', 'Cuidado', 'Cuidado', 'Compras', 'Salud', 'Personal', 'Personal', 'Otros'])

# Creo variable de modos de viajes agregada
viajes['Modo_tipo'] = viajes.tipo_viaje.replace([1,2,3,4], ['No Motorizado', 'Público', 'Privado', 'Mixto'])

viajes['Zona_destino'] = 'Destino Conurbano'
viajes.loc[viajes.bivi05_01=='CABA', 'Zona_destino'] = 'Destino CABA'

viajes['Zona_origen'] = 'Origen Conurbano'
viajes.loc[viajes.bivi04_01=='CABA', 'Zona_origen'] = 'Origen CABA'


viajes = pd.concat([viajes, pd.get_dummies(viajes.Modo_tipo)], axis=1)

# Incluyo factor de expansión y Zona 
personas = personas.set_index(['id_muestra', 'nro_vivienda', 'nro_hogar']).join(hogares[['id_muestra', 'nro_vivienda', 'nro_hogar', 'Zona']].set_index(['id_muestra', 'nro_vivienda', 'nro_hogar'])).reset_index()
viajes = viajes.set_index(['id_muestra', 'nro_vivienda', 'nro_hogar']).join(hogares[['id_muestra', 'nro_vivienda', 'nro_hogar', 'Zona', 'f_expansion']].set_index(['id_muestra', 'nro_vivienda', 'nro_hogar'])).reset_index()
etapas = etapas.set_index(['id_muestra', 'nro_vivienda', 'nro_hogar']).join(hogares[['id_muestra', 'nro_vivienda', 'nro_hogar', 'Zona', 'f_expansion']].set_index(['id_muestra', 'nro_vivienda', 'nro_hogar'])).reset_index()

In [3]:
(viajes[viajes.Motivo_agg!='Hogar'].groupby('Motivo_agg').f_expansion.sum().sort_values(ascending=False) / viajes[viajes.Motivo_agg!='Hogar'].f_expansion.sum()).round(2).sort_values(ascending=False)

Motivo_agg
Trabajo     0.43
Cuidado     0.15
Estudio     0.14
Personal    0.13
Compras     0.08
Salud       0.06
Otros       0.01
Name: f_expansion, dtype: float64

# Análisis de Etapas de viajes
La tabla de Etapas de viajes incluye todos los modos de transporte utilizados para un viaje

In [4]:
# agrupo por biet03 (modo de transporte)
print('Modos de transporte (Etapas)')
etapas1 = (etapas.groupby('biet03').f_expansion.sum()).astype(int).astype(int).reset_index()
etapas1['Porcentaje'] = (etapas1.f_expansion/etapas1.f_expansion.sum()*100).astype(int)
etapas1 = pd.concat([etapas1, DataFrame(['A pie','Colectivo','Vehículo motorizado (auto, moto, etc.) como conductor',
                                                       'Vehículo motorizado (auto, moto, etc.) como acompañante','Ferrocarril','Subte/Premetro','Bicicleta',
                                                       'Taxi','Remis','Transporte escolar','Charter/combi','Bus de la empresa donde trabaja','Otro'], columns=['Modo'])], axis=1)

etapas1 = pd.concat([etapas1, DataFrame(['No Motorizado','Público','Auto/Moto','Auto/Moto','Público','Público','No Motorizado','A demanda','A demanda',
                                                       'A demanda','A demanda','A demanda','A demanda'], columns=['Modo_agg'])], axis=1)

# Agrego descripción de modos a etapas
etapas = etapas.set_index('biet03').join(etapas1[['biet03', 'Modo', 'Modo_agg']].set_index('biet03')).sort_values('id_muestra').reset_index()

etapas1 = etapas1[['Modo', 'Modo_agg', 'f_expansion', 'Porcentaje']]
etapas1.columns = ['Modo', 'Modo_agg', 'Etapas', 'Etapas_porc']

#CABA
etapas1_caba = (etapas[etapas.Zona=='CABA'].groupby('biet03').f_expansion.sum()).astype(int).astype(int).reset_index()
etapas1_caba['CABA_porc'] = (etapas1_caba.f_expansion/etapas1_caba.f_expansion.sum()*100).astype(int)
etapas1_caba = pd.concat([etapas1_caba, DataFrame(['A pie','Colectivo','Vehículo motorizado (auto, moto, etc.) como conductor',
                                                       'Vehículo motorizado (auto, moto, etc.) como acompañante','Ferrocarril','Subte/Premetro','Bicicleta',
                                                       'Taxi','Remis','Transporte escolar','Charter/combi','Bus de la empresa donde trabaja','Otro'], columns=['Modo'])], axis=1)
etapas1_caba = etapas1_caba[['Modo', 'f_expansion', 'CABA_porc']]
etapas1_caba.columns = ['Modo', 'CABA', 'CABA_porc']

#AMBA
etapas1_amba = (etapas[etapas.Zona=='Conurbano'].groupby('biet03').f_expansion.sum()).astype(int).astype(int).reset_index()
etapas1_amba['Conurbano_porc'] = (etapas1_amba.f_expansion/etapas1_amba.f_expansion.sum()*100).astype(int)
etapas1_amba = pd.concat([etapas1_amba, DataFrame(['A pie','Colectivo','Vehículo motorizado (auto, moto, etc.) como conductor',
                                                       'Vehículo motorizado (auto, moto, etc.) como acompañante','Ferrocarril','Subte/Premetro','Bicicleta',
                                                       'Taxi','Remis','Transporte escolar','Charter/combi','Bus de la empresa donde trabaja','Otro'], columns=['Modo'])], axis=1)
etapas1_amba = etapas1_amba[['Modo', 'f_expansion', 'Conurbano_porc']]
etapas1_amba.columns = ['Modo', 'Conurbano', 'Conurbano_porc']


etapas1 = etapas1.set_index('Modo').join(etapas1_caba.set_index('Modo')).reset_index()
etapas1 = etapas1.set_index('Modo').join(etapas1_amba.set_index('Modo')).reset_index()
display(etapas1.sort_values('Etapas', ascending=False))

display(etapas1.groupby('Modo_agg').sum().reset_index().sort_values('Etapas', ascending=False))


Modos de transporte (Etapas)


,Modo,Modo_agg,Etapas,Etapas_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
1,Colectivo,Público,9078326,37,1862695,34,7215631,38
0,A pie,No Motorizado,5487974,22,1423676,26,4064298,21
2,"Vehículo motorizado (auto, moto, etc.) como co...",Auto/Moto,4037207,16,815233,15,3221974,17
3,"Vehículo motorizado (auto, moto, etc.) como ac...",Auto/Moto,2134948,8,396557,7,1738391,9
4,Ferrocarril,Público,1272688,5,99141,1,1173547,6
5,Subte/Premetro,Público,703210,2,461547,8,241663,1
6,Bicicleta,No Motorizado,640367,2,122655,2,517712,2
8,Remis,A demanda,306040,1,16032,0,290008,1
9,Transporte escolar,A demanda,183308,0,14836,0,168472,0
7,Taxi,A demanda,166341,0,125729,2,40612,0


,Modo_agg,Etapas,Etapas_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
3,Público,11054224,44,2423383,43,8630841,45
1,Auto/Moto,6172155,24,1211790,22,4960365,26
2,No Motorizado,6128341,24,1546331,28,4582010,23
0,A demanda,838170,1,187807,2,650363,1


### Etapas de viajes sin transporte no motorizado

In [5]:
print('Sin transporte no motorizado (Etapas)')
etapas2 = etapas1[etapas1.Modo_agg!='No Motorizado'].copy()
etapas2['Etapas_porc'] = (etapas2['Etapas'] / etapas2['Etapas'].sum() * 100).round().astype(int)
etapas2['CABA_porc'] = (etapas2['CABA'] / etapas2['CABA_porc'].sum() * 100).round().astype(int)
etapas2['CABA_porc'] = (etapas2['CABA'] / etapas2['CABA_porc'].sum() * 100).round().astype(int)
etapas2['Conurbano_porc'] = (etapas2['Conurbano'] / etapas2['Conurbano_porc'].sum() * 100).round().astype(int)
display(etapas2.sort_values('Etapas', ascending=False))
etapas2.groupby('Modo_agg').sum().sort_values('Etapas', ascending=False)

Sin transporte no motorizado (Etapas)


,Modo,Modo_agg,Etapas,Etapas_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
1,Colectivo,Público,9078326,50,1862695,33,7215631,10021710
2,"Vehículo motorizado (auto, moto, etc.) como co...",Auto/Moto,4037207,22,815233,14,3221974,4474964
3,"Vehículo motorizado (auto, moto, etc.) como ac...",Auto/Moto,2134948,12,396557,7,1738391,2414432
4,Ferrocarril,Público,1272688,7,99141,2,1173547,1629926
5,Subte/Premetro,Público,703210,4,461547,8,241663,335643
8,Remis,A demanda,306040,2,16032,0,290008,402789
9,Transporte escolar,A demanda,183308,1,14836,0,168472,233989
7,Taxi,A demanda,166341,1,125729,2,40612,56406
10,Charter/combi,A demanda,112553,1,22339,0,90214,125297
11,Bus de la empresa donde trabaja,A demanda,54660,0,7585,0,47075,65382


,Etapas,Etapas_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
Modo_agg,,,,,,
Público,11054224,61,2423383,43,8630841,11987279
Auto/Moto,6172155,34,1211790,21,4960365,6889396
A demanda,838170,5,187807,2,650363,903282


### Etapas de viajes de solo los viajes en transporte público

In [6]:
print('Solo Transporte Público (Etapas)')
etapas3 = etapas1[etapas1.Modo_agg=='Público'].copy()
etapas3['Etapas_porc'] = (etapas3['Etapas'] / etapas3['Etapas'].sum() * 100).round().astype(int)
etapas3['CABA_porc'] = (etapas3['CABA'] / etapas3['CABA'].sum() * 100).round().astype(int)
etapas3['Conurbano_porc'] = (etapas3['Conurbano'] / etapas3['Conurbano'].sum() * 100).round().astype(int)
display(etapas3.sort_values('Etapas', ascending=False))
etapas3.to_csv('Etapas_tpubl2014.csv', sep=';', encoding='latin1')

Solo Transporte Público (Etapas)


,Modo,Modo_agg,Etapas,Etapas_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
1,Colectivo,Público,9078326,82,1862695,77,7215631,84
4,Ferrocarril,Público,1272688,12,99141,4,1173547,14
5,Subte/Premetro,Público,703210,6,461547,19,241663,3


# Análisis de viajes
La tabla de viajes incluye todos los viajes realizados por cada una de las pesonas que viaja. 
Cuando el viaje incluye varios modos de transporte se categoriza como Público Multietapa (si el viaje incluye una etapa de transporte público) o Otros Multietapa cuando no incluye etapa de transporte público

In [7]:
viajes0 = pd.concat([etapas, pd.get_dummies(etapas.Modo), pd.get_dummies(etapas.Modo_agg)], axis=1)

viajes0 = viajes0.groupby(['id_muestra', 'nro_vivienda', 'nro_hogar', 'Zona', 'biet01']).agg({'f_expansion':'max', 'Modo':'max', 'Modo_agg':'max', 'A pie':'sum', 'Bicicleta':'sum', 'Bus de la empresa donde trabaja':'sum', 'Charter/combi':'sum', 
                                                                                            'Colectivo':'sum', 'Ferrocarril':'sum', 'Otro':'sum', 'Remis':'sum', 'Subte/Premetro':'sum', 'Taxi':'sum',
                                                                                            'Transporte escolar':'sum', 'Vehículo motorizado (auto, moto, etc.) como acompañante':'sum',
                                                                                            'Vehículo motorizado (auto, moto, etc.) como conductor':'sum', 'A demanda':'max',
                                                                                            'Auto/Moto':'max', 'No Motorizado':'max', 'Público':'max'}).reset_index()


viajes0['cantidad_modos'] = viajes0[['A pie', 'Bicicleta',
       'Bus de la empresa donde trabaja', 'Charter/combi', 'Colectivo',
       'Ferrocarril', 'Otro', 'Remis', 'Subte/Premetro', 'Taxi',
       'Transporte escolar',
       'Vehículo motorizado (auto, moto, etc.) como acompañante',
       'Vehículo motorizado (auto, moto, etc.) como conductor']].sum(axis=1)
viajes0['cantidad_modosxtipo'] = viajes0[['A demanda','Auto/Moto', 'No Motorizado', 'Público']].sum(axis=1)


viajes0.loc[(viajes0.cantidad_modos>1), 'Modo'] = ''
viajes0.loc[(viajes0.cantidad_modos>1)&(viajes0.Público==1), 'Modo'] = 'Público Multietapa'
viajes0.loc[(viajes0.cantidad_modos>1)&(viajes0.Público!=1), 'Modo'] = 'Otros Multietapa'

viajes0.loc[(viajes0.cantidad_modos>1)&(viajes0.Público==1), 'Modo_agg'] = 'Público Multietapa'
viajes0.loc[(viajes0.cantidad_modos>1)&(viajes0.Público!=1), 'Modo_agg'] = 'Otros Multietapa'


viajes1 = viajes0.groupby(['Zona', 'Modo', 'Modo_agg']).f_expansion.sum().round(1).sort_values(ascending=False).reset_index()

viajes1['Modo_agg2'] = viajes1.Modo_agg.replace(['Público', 'No Motorizado', 'Auto/Moto', 'Público Multietapa',
                               'A demanda', 'Otros Multietapa'], ['Público', 'No Motorizado', 'Auto/Moto', 'Público', 'A demanda', 'A demanda'])


viajes1 = pd.pivot_table(data=viajes1, index=['Modo', 'Modo_agg', 'Modo_agg2'], columns='Zona', values='f_expansion').reset_index()
viajes1['Viajes'] = viajes1['CABA']+viajes1['Conurbano']

viajes1['Viajes_porc'] = (viajes1['Viajes'] / viajes1['Viajes'].sum() * 100).round(1)
viajes1['CABA_porc'] = (viajes1['CABA'] / viajes1['CABA'].sum() * 100).round(1)
viajes1['Conurbano_porc'] = (viajes1['Conurbano'] / viajes1['Conurbano'].sum() * 100).round(1)

viajes1 = viajes1[['Modo', 'Modo_agg', 'Modo_agg2', 'Viajes', 'Viajes_porc', 'CABA', 'CABA_porc', 'Conurbano',
        'Conurbano_porc']].sort_values('Viajes', ascending=False)


viajes1['Modo'] = viajes1.Modo.replace(['Colectivo', 'Público Multietapa', 'Subte/Premetro', 'Ferrocarril'],
                                                 ['Colectivo', 'Multietapa', 'Subte/Premetro', 'Ferrocarril'])





viajes1.to_csv('modos2014.csv', sep=';', encoding='latin1')

viajes1

Zona,Modo,Modo_agg,Modo_agg2,Viajes,Viajes_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
4,Colectivo,Público,Público,5921540.0,27.4,1507620.0,29.7,4413920.0,26.7
0,A pie,No Motorizado,No Motorizado,5487974.0,25.4,1423676.0,28.1,4064298.0,24.6
14,"Vehículo motorizado (auto, moto, etc.) como co...",Auto/Moto,Auto/Moto,4020148.0,18.6,809415.0,16.0,3210733.0,19.4
8,Multietapa,Público Multietapa,Público,2104287.0,9.7,281576.0,5.6,1822711.0,11.0
13,"Vehículo motorizado (auto, moto, etc.) como ac...",Auto/Moto,Auto/Moto,2068005.0,9.6,382748.0,7.5,1685257.0,10.2
1,Bicicleta,No Motorizado,No Motorizado,615513.0,2.8,119473.0,2.4,496040.0,3.0
10,Subte/Premetro,Público,Público,337558.0,1.6,329850.0,6.5,7708.0,0.0
5,Ferrocarril,Público,Público,323435.0,1.5,43515.0,0.9,279920.0,1.7
9,Remis,A demanda,A demanda,273486.0,1.3,13295.0,0.3,260191.0,1.6
12,Transporte escolar,A demanda,A demanda,177898.0,0.8,14143.0,0.3,163755.0,1.0


### Viajes motorizados por modo

In [8]:
print('Viajes por modo (solo motorizado)')
viajes2 = viajes1[viajes1.Modo_agg2!='No Motorizado'].copy()
viajes2['Viajes_porc'] = (viajes2['Viajes'] / viajes2['Viajes'].sum() * 100).round(2)
viajes2['CABA_porc'] = (viajes2['CABA'] / viajes2['CABA'].sum() * 100).round(2)
viajes2['Conurbano_porc'] = (viajes2['Conurbano'] / viajes2['Conurbano'].sum() * 100).round(2)
viajes2.groupby('Modo_agg').sum().sort_values('Viajes', ascending=False)

Viajes por modo (solo motorizado)


Zona,Viajes,Viajes_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
Modo_agg,,,,,,
Público,6582533.0,42.44,1880985.0,53.33,4701548.0,39.23
Auto/Moto,6088153.0,39.25,1192163.0,33.80,4895990.0,40.85
Público Multietapa,2104287.0,13.57,281576.0,7.98,1822711.0,15.21
A demanda,723272.0,4.67,171484.0,4.87,551788.0,4.61
Otros Multietapa,14153.0,0.09,688.0,0.02,13465.0,0.11


### Viajes en transporte público por modo

In [9]:
print('Viajes por modo (transporte público)')
viajes3 = viajes1[viajes1.Modo_agg2=='Público'].copy()
viajes3['Viajes_porc'] = (viajes3['Viajes'] / viajes3['Viajes'].sum() * 100).round(1)
viajes3['CABA_porc'] = (viajes3['CABA'] / viajes3['CABA'].sum() * 100).round(1)
viajes3['Conurbano_porc'] = (viajes3['Conurbano'] / viajes3['Conurbano'].sum() * 100).round(1)
viajes3 = viajes3.groupby('Modo').sum().sort_values('Viajes', ascending=False)

viajes3.to_csv('modos_publico2014.csv', sep=';', encoding='latin1')
viajes3

Viajes por modo (transporte público)


Zona,Viajes,Viajes_porc,CABA,CABA_porc,Conurbano,Conurbano_porc
Modo,,,,,,
Colectivo,5921540.0,68.2,1507620.0,69.7,4413920.0,67.7
Multietapa,2104287.0,24.2,281576.0,13.0,1822711.0,27.9
Subte/Premetro,337558.0,3.9,329850.0,15.3,7708.0,0.1
Ferrocarril,323435.0,3.7,43515.0,2.0,279920.0,4.3


# Construcción de indicadores para el análisis

In [10]:
# Población
datos = DataFrame([['Población', personas.f_expansion.sum().round().astype(int), personas[personas.Zona=='Conurbano'].f_expansion.sum().round().astype(int), 
                    personas[personas.Zona=='CABA'].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA'])


# Viajes
datos = datos.append(DataFrame([['Viajes (partido hogar)', viajes.f_expansion.sum().round().astype(int), 
                                 viajes[viajes.Zona=='Conurbano'].f_expansion.sum().round().astype(int), 
                                 viajes[viajes.Zona=='CABA'].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(DataFrame([['Viajes (partido origen)', viajes.f_expansion.sum().round().astype(int), 
                                 viajes[viajes.Zona_origen=='Origen Conurbano'].f_expansion.sum().round().astype(int), 
                                 viajes[viajes.Zona_origen=='Origen CABA'].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(DataFrame([['Viajes (partido destino)', viajes.f_expansion.sum().round().astype(int), 
                                 viajes[viajes.Zona_destino=='Destino Conurbano'].f_expansion.sum().round().astype(int), 
                                 viajes[viajes.Zona_destino=='Destino CABA'].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))


datos = datos.append(DataFrame([['Viajes en TPubl (partido hogar)', viajes[(viajes.Público==1)].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona=='Conurbano')&(viajes.Público==1)].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona=='CABA')&(viajes.Público==1)].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(DataFrame([['Viajes en TPubl (partido origen)', viajes[(viajes.Público==1)].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona_origen=='Origen Conurbano')&(viajes.Público==1)].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona_origen=='Origen CABA')&(viajes.Público==1)].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(DataFrame([['Viajes en TPubl (partido destino)', viajes[(viajes.Público==1)].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona_destino=='Destino Conurbano')&(viajes.Público==1)].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona_destino=='Destino CABA')&(viajes.Público==1)].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))


# Viajes (sin vuelta a hogar)
datos = datos.append(DataFrame([['Viajes (partido hogar) (sin vuelta Hogar)', viajes[(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona=='Conurbano')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona=='CABA')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(DataFrame([['Viajes (partido origen) (sin vuelta Hogar)', viajes[(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona_origen=='Origen Conurbano')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona_origen=='Origen CABA')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(DataFrame([['Viajes (partido destino) (sin vuelta Hogar)', viajes[(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona_destino=='Destino Conurbano')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona_destino=='Destino CABA')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(DataFrame([['Viajes en TPubl (partido hogar) (sin vuelta Hogar)', viajes[(viajes.Público==1)&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona=='Conurbano')&(viajes.Público==1)&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona=='CABA')&(viajes.Público==1)&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(DataFrame([['Viajes en TPubl (partido origen) (sin vuelta Hogar)', viajes[(viajes.Público==1)&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona_origen=='Origen Conurbano')&(viajes.Público==1)&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona_origen=='Origen CABA')&(viajes.Público==1)&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(DataFrame([['Viajes en TPubl (partido destino) (sin vuelta Hogar)', viajes[(viajes.Público==1)&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona_destino=='Destino Conurbano')&(viajes.Público==1)&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                 viajes[(viajes.Zona_destino=='Destino CABA')&(viajes.Público==1)&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

# Etapas
datos = datos.append(DataFrame([['Etapas', etapas.f_expansion.sum().round().astype(int), etapas[etapas.Zona=='Conurbano'].f_expansion.sum().round().astype(int), 
                                 etapas[etapas.Zona=='CABA'].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(DataFrame([['Etapas TPubl', etapas[(etapas.Modo_agg =='Público')].f_expansion.sum().round().astype(int), 
                                 etapas[(etapas.Zona=='Conurbano')&(etapas.Modo_agg =='Público')].f_expansion.sum().round().astype(int), 
                                 etapas[(etapas.Zona=='CABA')&(etapas.Modo_agg =='Público')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

# Viajes promedio por persona (promedio ponderado por fex)
pers_temp = viajes.groupby(['id_muestra', 'nro_vivienda', 'nro_hogar', 'bivi01', 'Zona', 'f_expansion']).size().reset_index()
pers_temp['pond'] = pers_temp['f_expansion'] * pers_temp[0]
datos = datos.append(DataFrame([['Viajes por persona (promedio)', (pers_temp.pond.sum() / pers_temp.f_expansion.sum()).round(2), 
            (pers_temp[pers_temp.Zona=='Conurbano'].pond.sum() / pers_temp[pers_temp.Zona=='Conurbano'].f_expansion.sum()).round(2), 
            (pers_temp[pers_temp.Zona=='CABA'].pond.sum() / pers_temp[pers_temp.Zona=='CABA'].f_expansion.sum()).round(2)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

# Viajes promedio por persona en TPubl (promedio ponderado por fex)
pers_temp = viajes[viajes.Público==1].groupby(['id_muestra', 'nro_vivienda', 'nro_hogar', 'bivi01', 'Zona', 'f_expansion']).size().reset_index()
pers_temp['pond'] = pers_temp['f_expansion'] * pers_temp[0]
datos = datos.append(DataFrame([['Viajes por persona en TPubl (promedio)', (pers_temp.pond.sum() / pers_temp.f_expansion.sum()).round(2), 
            (pers_temp[pers_temp.Zona=='Conurbano'].pond.sum() / pers_temp[pers_temp.Zona=='Conurbano'].f_expansion.sum()).round(2), 
            (pers_temp[pers_temp.Zona=='CABA'].pond.sum() / pers_temp[pers_temp.Zona=='CABA'].f_expansion.sum()).round(2)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))


# Personas que viajan (según la zona del hogar)
pers_temp = viajes.groupby(['id_muestra', 'nro_vivienda', 'nro_hogar', 'bivi01', 'Zona', 'f_expansion']).agg({'Mixto':'max', 'No Motorizado':'max', 'Privado':'max', 'Público':'max'}).reset_index()
datos = datos.append(DataFrame([['Personas que viajan (hogar)', pers_temp.f_expansion.sum().round().astype(int), 
            pers_temp[pers_temp.Zona=='Conurbano'].f_expansion.sum().round().astype(int), 
            pers_temp[pers_temp.Zona=='CABA'].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

pers_temp2 = viajes.groupby(['id_muestra', 'nro_vivienda', 'nro_hogar', 'bivi01', 'Zona_origen', 'f_expansion']).agg({'Mixto':'max', 'No Motorizado':'max', 'Privado':'max', 'Público':'max'}).reset_index()
datos = datos.append(DataFrame([['Personas que viajan (origen)', pers_temp2.f_expansion.sum().round().astype(int), 
            pers_temp2[pers_temp2.Zona_origen=='Origen Conurbano'].f_expansion.sum().round().astype(int), 
            pers_temp2[pers_temp2.Zona_origen=='Origen CABA'].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

# Personas que viajan (construido desde viajes - viajes/promedio de viajes por persona)
datos = datos.append(pd.DataFrame([['Pers q viajan (hogar-dde viajes)', (datos[datos.dato=='Viajes (partido hogar)'].Total.values[0] / datos[datos.dato=='Viajes por persona (promedio)'].Total.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes (partido hogar)'].Conurbano.values[0] / datos[datos.dato=='Viajes por persona (promedio)'].Conurbano.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes (partido hogar)'].CABA.values[0] / datos[datos.dato=='Viajes por persona (promedio)'].CABA.values[0]).round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Pers q viajan (origen-dde viajes)', (datos[datos.dato=='Viajes (partido origen)'].Total.values[0] / datos[datos.dato=='Viajes por persona (promedio)'].Total.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes (partido origen)'].Conurbano.values[0] / datos[datos.dato=='Viajes por persona (promedio)'].Conurbano.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes (partido origen)'].CABA.values[0] / datos[datos.dato=='Viajes por persona (promedio)'].CABA.values[0]).round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(DataFrame([['Personas que viajan en TPubl (hogar)', pers_temp[(pers_temp['Público']==1)].f_expansion.sum().round().astype(int), 
            pers_temp[(pers_temp.Zona=='Conurbano')&(pers_temp['Público']==1)].f_expansion.sum().round().astype(int), 
            pers_temp[(pers_temp.Zona=='CABA')&(pers_temp['Público']==1)].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(DataFrame([['Personas que viajan en TPubl (origen)', pers_temp2[(pers_temp2['Público']==1)].f_expansion.sum().round().astype(int), 
            pers_temp2[(pers_temp2.Zona_origen=='Origen Conurbano')&(pers_temp2['Público']==1)].f_expansion.sum().round().astype(int), 
            pers_temp2[(pers_temp2.Zona_origen=='Origen CABA')&(pers_temp2['Público']==1)].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Pers q viajan TP (hogar-dde viajes)', (datos[datos.dato=='Viajes en TPubl (partido hogar)'].Total.values[0] / datos[datos.dato=='Viajes por persona en TPubl (promedio)'].Total.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes en TPubl (partido hogar)'].Conurbano.values[0] / datos[datos.dato=='Viajes por persona en TPubl (promedio)'].Conurbano.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes en TPubl (partido hogar)'].CABA.values[0] / datos[datos.dato=='Viajes por persona en TPubl (promedio)'].CABA.values[0]).round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Pers q viajan TP (origen-dde viajes)', (datos[datos.dato=='Viajes en TPubl (partido origen)'].Total.values[0] / datos[datos.dato=='Viajes por persona en TPubl (promedio)'].Total.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes en TPubl (partido origen)'].Conurbano.values[0] / datos[datos.dato=='Viajes por persona en TPubl (promedio)'].Conurbano.values[0]).round().astype(int), \
        (datos[datos.dato=='Viajes en TPubl (partido origen)'].CABA.values[0] / datos[datos.dato=='Viajes por persona en TPubl (promedio)'].CABA.values[0]).round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(DataFrame([['Trabajadores', personas[personas.bhmo03==1].f_expansion.sum().round().astype(int), 
                                 personas[(personas.Zona=='Conurbano')&(personas.bhmo03==1)].f_expansion.sum().round().astype(int), 
                                 personas[(personas.Zona=='CABA')&(personas.bhmo03==1)].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))


# Rate Viajes/Personas
v1 = viajes[viajes.Motivo_agg=='Trabajo'].groupby(['id_muestra', 'nro_vivienda', 'nro_hogar', 'bivi01', 'f_expansion']).size().reset_index()
v1['tot_temp'] = v1['f_expansion'] * v1[0]
v1 = (v1.f_expansion.sum() / v1.tot_temp.sum() ).round(2)
datos = datos.append(pd.DataFrame([['Rate Viajes/Persona (Trabajo)', v1, v1, v1]], columns=['dato', 'Total', 'Conurbano', 'CABA'] ))

v1 = viajes[(viajes.Motivo_agg!='Hogar')&(viajes.Motivo_agg!='Trabajo')].groupby(['id_muestra', 'nro_vivienda', 'nro_hogar', 'bivi01', 'f_expansion']).size().reset_index()
v1['tot_temp'] = v1['f_expansion'] * v1[0]
v1 = (v1.f_expansion.sum() / v1.tot_temp.sum()).round(2)
datos = datos.append(pd.DataFrame([['Rate Viajes/Persona (Otros)', v1, v1, v1]], columns=['dato', 'Total', 'Conurbano', 'CABA'] ))


# Construyo matriz de origen y destino de viajes total, motivo trabajo y motivo otros
datos = datos.append(pd.DataFrame([['Viajes de CABA a', viajes[viajes.Zona=='CABA'].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='CABA')&(viajes.Zona_destino=='Destino Conurbano')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='CABA')&(viajes.Zona_destino=='Destino CABA')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(pd.DataFrame([['Viajes de Conurbano a', viajes[viajes.Zona=='Conurbano'].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='Conurbano')&(viajes.Zona_destino=='Destino Conurbano')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='Conurbano')&(viajes.Zona_destino=='Destino CABA')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Viajes de CABA a (sin vuelta Hogar)', viajes[(viajes.Zona=='CABA')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='CABA')&(viajes.Zona_destino=='Destino Conurbano')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='CABA')&(viajes.Zona_destino=='Destino CABA')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(pd.DataFrame([['Viajes de Conurbano a (sin vuelta Hogar)', viajes[(viajes.Zona=='Conurbano')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='Conurbano')&(viajes.Zona_destino=='Destino Conurbano')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='Conurbano')&(viajes.Zona_destino=='Destino CABA')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Viajes trabajo de CABA a', viajes[(viajes.Zona=='CABA')&(viajes['Motivo_agg']=='Trabajo')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='CABA')&(viajes.Zona_destino=='Destino Conurbano')&(viajes['Motivo_agg']=='Trabajo')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='CABA')&(viajes.Zona_destino=='Destino CABA')&(viajes['Motivo_agg']=='Trabajo')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(pd.DataFrame([['Viajes trabajo de Conurbano a', viajes[(viajes.Zona=='Conurbano')&(viajes['Motivo_agg']=='Trabajo')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='Conurbano')&(viajes.Zona_destino=='Destino Conurbano')&(viajes['Motivo_agg']=='Trabajo')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='Conurbano')&(viajes.Zona_destino=='Destino CABA')&(viajes['Motivo_agg']=='Trabajo')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Viajes otros de CABA a', viajes[(viajes.Zona=='CABA')&(viajes['Motivo_agg']!='Trabajo')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='CABA')&(viajes.Zona_destino=='Destino Conurbano')&(viajes['Motivo_agg']!='Trabajo')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='CABA')&(viajes.Zona_destino=='Destino CABA')&(viajes['Motivo_agg']!='Trabajo')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))
datos = datos.append(pd.DataFrame([['Viajes otros de Conurbano a', viajes[(viajes.Zona=='Conurbano')&(viajes['Motivo_agg']!='Trabajo')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='Conurbano')&(viajes.Zona_destino=='Destino Conurbano')&(viajes['Motivo_agg']!='Trabajo')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='Conurbano')&(viajes.Zona_destino=='Destino CABA')&(viajes['Motivo_agg']!='Trabajo')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))


datos = datos.append(pd.DataFrame([['Viajes otros de CABA a (sin vuelta Hogar)', viajes[(viajes.Zona=='CABA')&(viajes['Motivo_agg']!='Trabajo')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='CABA')&(viajes.Zona_destino=='Destino Conurbano')&(viajes['Motivo_agg']!='Trabajo')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='CABA')&(viajes.Zona_destino=='Destino CABA')&(viajes['Motivo_agg']!='Trabajo')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))

datos = datos.append(pd.DataFrame([['Viajes otros de Conurbano a (sin vuelta Hogar)', viajes[(viajes.Zona=='Conurbano')&(viajes['Motivo_agg']!='Trabajo')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='Conurbano')&(viajes.Zona_destino=='Destino Conurbano')&(viajes['Motivo_agg']!='Trabajo')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int), 
                                    viajes[(viajes.Zona=='Conurbano')&(viajes.Zona_destino=='Destino CABA')&(viajes['Motivo_agg']!='Trabajo')&(viajes['Motivo_agg']!='Hogar')].f_expansion.sum().round().astype(int)]], columns=['dato', 'Total', 'Conurbano', 'CABA']))




# BHVM02_01 - Cantidad de autos (totales en hogares)
hogares['bhvm02_01'] = hogares['bhvm02_01'].fillna(0)
hogares['Hog_Cant_autos'] = hogares['bhvm02_01'] * hogares['f_expansion']
(hogares.Hog_Cant_autos.sum()).round(2), \
(hogares.Hog_Cant_autos.sum() / hogares.f_expansion.sum()).round(2)

datos = datos.append(pd.DataFrame([['Cantidad de autos (totales en hogares)', (hogares.Hog_Cant_autos.sum()).round(2), 
                                    (hogares[hogares.Zona=='Conurbano'].Hog_Cant_autos.sum()).round(2), 
                                    (hogares[hogares.Zona=='CABA'].Hog_Cant_autos.sum()).round(2),]], columns=['dato', 'Total', 'Conurbano', 'CABA']))




# Construyo campos de porcentajes 
datos['Conurbano_porc'] = (datos['Conurbano'] / datos['Total'] * 100).round(2)
datos['CABA_porc'] = (datos['CABA'] / datos['Total'] * 100).round(2)

datos['Total_porc_pobl'] = (datos.Total / datos[datos.dato=='Población'].Total.values * 100).round(2)
datos['Conurbano_porc_pobl'] = (datos.Conurbano / datos[datos.dato=='Población'].Conurbano.values * 100).round(2)
datos['CABA_porc_pobl'] = (datos.CABA / datos[datos.dato=='Población'].CABA.values * 100).round(2)


datos = datos.fillna(0)


datos = datos.reset_index(drop=True)
datos.to_csv('datos_2014.csv', sep=';', encoding='latin1')
datos

,dato,Total,Conurbano,CABA,Conurbano_porc,CABA_porc,Total_porc_pobl,Conurbano_porc_pobl,CABA_porc_pobl
0,Población,14422940.00,11562732.00,2860208.00,80.17,19.83,100.00,100.00,100.00
1,Viajes (partido hogar),10009259.00,7596592.00,2412667.00,75.90,24.10,69.40,65.70,84.35
2,Viajes (partido origen),10009259.00,7184734.00,2824525.00,71.78,28.22,69.40,62.14,98.75
3,Viajes (partido destino),10009259.00,7003404.00,3005855.00,69.97,30.03,69.40,60.57,105.09
4,Viajes en TPubl (partido hogar),4103227.00,3077437.00,1025790.00,75.00,25.00,28.45,26.62,35.86
5,Viajes en TPubl (partido origen),4103227.00,2834290.00,1268937.00,69.07,30.93,28.45,24.51,44.37
6,Viajes en TPubl (partido destino),4103227.00,2721825.00,1381402.00,66.33,33.67,28.45,23.54,48.30
7,Viajes (partido hogar) (sin vuelta Hogar),6315913.00,4693682.00,1622231.00,74.32,25.68,43.79,40.59,56.72
8,Viajes (partido origen) (sin vuelta Hogar),6315913.00,4662579.00,1653334.00,73.82,26.18,43.79,40.32,57.80
9,Viajes (partido destino) (sin vuelta Hogar),6315913.00,4102677.00,2213236.00,64.96,35.04,43.79,35.48,77.38


In [11]:
viajes.head(2)

,id_muestra,nro_vivienda,nro_hogar,bivi01,bivi02,bivi03,bivi04_01,bivi04_02,bivi04_03,bivi04_04,bivi04_05,bivi05_01,bivi05_02,bivi05_03,bivi05_04,bivi05_05,bivi06_01,bivi06_02,bivi07,bivi08,bivi09,tipo_viaje,origen_latitud,origen_longitud,destino_latitud,destino_longitud,Motivo,Motivo_agg,Modo_tipo,Zona_destino,Zona_origen,Mixto,No Motorizado,Privado,Público,Zona,f_expansion
0,146684,1,1,1,1,02/12,BERISSO,H,HOGAR,NaN,NaN,LA PLATA,C,37,E,AV. 7,1,2,07:40,09:00,6,2,-34.89871,-57.91151,-34.90477,-57.96327,Lugar de trabajo,Trabajo,Público,Destino Conurbano,Origen Conurbano,0,0,0,1,Conurbano,397.0
1,616429,1,1,1,1,23/09,MORENO,H,HOGAR,NaN,NaN,VICENTE LOPEZ,H,HOSPITAL DR. B. HOUSSAY,NaN,NaN,1,10,07:30,11:30,1,2,-34.60522,-58.79516,-34.54195,-58.52828,Salud,Salud,Público,Destino Conurbano,Origen Conurbano,0,0,0,1,Conurbano,323.0


In [12]:
v = viajes[viajes.Modo_tipo!='No Motorizado'].groupby(['Zona', 'Zona_destino', 'Modo_tipo']).f_expansion.sum().round().reset_index()
v['tot'] = v.groupby(['Zona', 'Zona_destino']).f_expansion.transform('sum') 
v['tot'] = (v['f_expansion'] / v['tot'] * 100).round(1)
v[(v.Modo_tipo=='Público')|(v.Modo_tipo=='Mixto')].groupby(['Zona', 'Zona_destino']).sum()


f_expansion   tot
Zona      Zona_destino                        
CABA      Destino CABA          988145.0  63.0
          Destino Conurbano      63903.0  54.3
Conurbano Destino CABA          436262.0  64.1
          Destino Conurbano    2738363.0  56.4

In [13]:
v.Modo_tipo.unique()

array(['Mixto', 'Privado', 'Público'], dtype=object)

In [14]:
# # Origen y destino de los viajes por trabajo
# (viajes[viajes['Motivo_agg']=='Trabajo'].groupby(['Zona', 'Zona_destino']).f_expansion.sum().round() / viajes[viajes['Motivo_agg']=='Trabajo'].groupby(['Zona', 'Zona_destino']).f_expansion.sum().round().sum() * 100 ).round().reset_index()

viajes['Trabajo']=0
viajes['Otros']=0
viajes.loc[viajes.Motivo_agg=='Trabajo', 'Trabajo'] = 1
viajes.loc[viajes.Motivo_agg!='Trabajo', 'Otros'] = 1

pers_temp3 = viajes.groupby(['id_muestra', 'nro_vivienda', 'nro_hogar', 'bivi01', 'Zona', 'Zona_destino', 'f_expansion']).agg({'Trabajo':'max'}).reset_index()
pers_temp3[(pers_temp3.Zona=='Conurbano')&(pers_temp3.Trabajo==1)].f_expansion.sum()


1808522.0

# Motivos de los viajes

In [15]:
dest = viajes.groupby('Motivo_agg').f_expansion.sum().round().astype(int).reset_index()
dest['porc_tot'] = (dest['f_expansion'] / dest['f_expansion'].sum() * 100).round(1)
dest['porc_tot_sinHogar'] = 0 
dest.loc[dest.Motivo_agg!='Hogar', 'porc_tot_sinHogar'] = (dest.loc[dest.Motivo_agg!='Hogar', 'f_expansion'] / dest.loc[dest.Motivo_agg!='Hogar', 'f_expansion'].sum() * 100).round(1)

dest = dest.rename(columns={'f_expansion': 'Total', 'Motivo_agg': 'Motivo'})

dest['Total_porc_pobl'] = (dest.Total / datos[datos.dato=='Población'].Total.values * 100).round(2)

dest.to_csv('motivos2014.csv', sep=';')
dest.sort_values('porc_tot_sinHogar', ascending=False)

,Motivo,Total,porc_tot,porc_tot_sinHogar,Total_porc_pobl
7,Trabajo,2726621,27.2,43.2,18.90
1,Cuidado,948187,9.5,15.0,6.57
2,Estudio,910633,9.1,14.4,6.31
5,Personal,833031,8.3,13.2,5.78
0,Compras,489112,4.9,7.7,3.39
6,Salud,352673,3.5,5.6,2.45
4,Otros,55656,0.6,0.9,0.39
3,Hogar,3693346,36.9,0.0,25.61


In [16]:
# Viajes por motivo
viajes_motivo = viajes.pivot_table('f_expansion', 'Motivo_agg', 'Zona', aggfunc='sum').reset_index()
viajes_motivo['AMBA'] = viajes_motivo['CABA'] + viajes_motivo['Conurbano']
viajes_motivo = viajes_motivo.sort_values('AMBA', ascending=False)
viajes_motivo = viajes_motivo[['AMBA', 'Conurbano', 'CABA']]
viajes_motivo

Zona,AMBA,Conurbano,CABA
3,3693346.0,2902910.0,790436.0
7,2726621.0,1975743.0,750878.0
1,948187.0,741285.0,206902.0
2,910633.0,748068.0,162565.0
5,833031.0,580948.0,252083.0
0,489112.0,348673.0,140439.0
6,352673.0,256196.0,96477.0
4,55656.0,42769.0,12887.0


In [17]:
# Promedio de viajes por persona (motivo trabajo, otros, total sin hogar, total)
viajes[viajes.Motivo_agg=='Trabajo'].groupby(['id_muestra', 'nro_vivienda', 'nro_hogar', 'bivi01']).size().mean().round(2), \
viajes[(viajes.Motivo_agg!='Trabajo')&(viajes.Motivo_agg!='Hogar')].groupby(['id_muestra', 'nro_vivienda', 'nro_hogar', 'bivi01']).size().mean().round(2), \
viajes[(viajes.Motivo_agg!='Hogar')].groupby(['id_muestra', 'nro_vivienda', 'nro_hogar', 'bivi01']).size().mean().round(2), \
viajes.groupby(['id_muestra', 'nro_vivienda', 'nro_hogar', 'bivi01']).size().mean().round(2)

(1.1, 1.12, 1.11, 1.11)

# Vehículos

In [18]:
# Tiene Auto (si/no)
hogares[hogares.bhvm02==1].f_expansion.sum().round().astype(int), \
hogares[hogares.bhvm02==1].f_expansion.sum() / hogares.f_expansion.sum().round(2)

(2181599, 0.4611336800010653)

In [19]:
# Tiene moto/ciclomotor (si/no)
(hogares[hogares.bhmc01==1].f_expansion.sum()).round().astype(int), \
(hogares[hogares.bhmc01==1].f_expansion.sum() / hogares.f_expansion.sum()).round(2)


(445970, 0.09)

In [20]:
# Tiene auto o moto (si/no)
hogares[(hogares.bhvm02==1)|(hogares.bhmc01==1)].f_expansion.sum().round().astype(int), \
(hogares[(hogares.bhvm02==1)|(hogares.bhmc01==1)].f_expansion.sum() / hogares.f_expansion.sum()).round(2)

(2408982, 0.51)

In [21]:
# BHVM02_01 - Cantidad de autos (totales en hogares)
hogares['bhvm02_01'] = hogares['bhvm02_01'].fillna(0)
hogares['Hog_Cant_autos'] = hogares['bhvm02_01'] * hogares['f_expansion']
(hogares.Hog_Cant_autos.sum()).round(2), \
(hogares.Hog_Cant_autos.sum() / hogares.f_expansion.sum()).round(2)

(2497812.0, 0.53)

In [22]:
# Estimado cantidad de autos de uso personal
# (enmodmo 2009) %39 y (enmodo 2014) 53% de hogares con automovil - Hogares en AMBA: 5137277 (EPH 3T2019)
5137277 * .39, 5137277 * .53

(2003538.03, 2722756.81)

In [23]:
# Tasa de crecimiento vehicular (CABA y AMBA)
tasa_caba_amba = 0.0442
autos_2009 = 1621245
autos_2014 = 0

autos = pd.DataFrame([[2009, autos_2009, 0]], columns=['Año', '2009', '2014'])
for i in [2010, 2011, 2013, 2014, 2015, 2016, 2017, 2018, 2019]:
           
    autos_2009 = int(round(autos_2009+(autos_2009*tasa_caba_amba)))
    
    if i == 2014: autos_2014 = 2408982
    if i > 2014: autos_2014 = int(round(autos_2014+(autos_2014*tasa_caba_amba)))
        
    autos = autos.append(pd.DataFrame([[i, autos_2009, autos_2014]], columns=['Año', '2009', '2014']))
    
    
autos['Año'] = autos['Año'].astype(str)
autos.to_csv('parque_vehicular.csv', sep=';')
autos

,Año,2009,2014
0,2009,1621245,0
0,2010,1692904,0
0,2011,1767730,0
0,2013,1845864,0
0,2014,1927451,2408982
0,2015,2012644,2515459
0,2016,2101603,2626642
0,2017,2194494,2742740
0,2018,2291491,2863969
0,2019,2392775,2990556
